# Cao, Chen, Hull, Poulos, 2019 - Deep Hedging of Derivatives Using Reinforcement Learning

This notebook proposes an implementation of the concepts described in the article mentionned above.

**For now, only the parameters to to try and reproduce Table 1 are used**

*Option parameters*
- Short call option
- Maturity: 1 month
- Strike: At the money
- Trading cost: 1%

*Underlying parameters*
- Geometric Brownian motion
- Volatility: 20%
- Expected return: 5%
- Dividend yield: 0%
- Risk-free rate: 0%



**Objective function**

$$ Y(t) = \mathbb{E}[C_t] + c \cdot \sqrt{ \mathbb{E}[C_t^2] - \mathbb{E}[C_t]^2 }$$
where:
- $C_t$ is the total hedging cost from time $t$ onward
- $\mathbb{E}[C_t]$ is the expected hedging cost
- $c$ is a constant

The objective is to minimize $Y(0)$.
Y is a coherent risk measure and satisfies Bellman equation.

# Imports

In [5]:
!pip install gymnasium

import datetime as dt
import gymnasium as gym
from gymnasium import spaces # https://gymnasium.farama.org/api/spaces/
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.random as sim
import random
import scipy.stats as stats
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm

random.seed(2023)
np.random.seed(2023)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Files

In [19]:
# import os
# from google.colab import drive

# WORK_DRIVE = '/gdrive'
# WORK_AREA = WORK_DRIVE + '/MyDrive/Cao and al, 2019 - Deep Hedging of Derivatives using Reinforcement Learning'
# drive.mount(WORK_DRIVE)
# os.chdir(WORK_AREA)

# if not os.path.isdir('Cao and al, 2019 - models'):
#     os.makedirs('Cao and al, 2019 - models')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Preliminaries

In the following we will use the below convention:
- Number of days in a year: 252
- Number of days in a month: 21

## Geometric brownian motion
$$ dS = \mu S dt + \sigma S dBt $$

*Euler scheme*
$$ S_{t+1} = S_t + \mu S_t \Delta t + S_t \sigma \Delta B_t $$
where:
- $\Delta B_t = \sqrt{\Delta t} \cdot G$
- $ G \sim \mathcal{N}(0,1)$

In [7]:
def simulate_geometric_brownian_motion(S0, mu, sigma, n, N, dt=1/252, plot=False):
    """
    Parameters
    ----------
    S0: float
        Initial underlying price
    mu: float
        Expected return
    sigma: float
        Volatility
    n: int
        Number of increments (in days, convention is 252 per year)
    N: int
        Number of trajectory
    dt: float
        Size of increment (convention is 1/252)
    """

    S = np.ones((n+1, N)) * S0

    for j in range(N) :
        for i in range(1, n+1) :
            S[i,j] = S[i-1,j] * (1 + mu*dt + sigma*np.sqrt(dt)*sim.randn())

    if plot:
        plot_process(S)
        plt.show()

    return S

def plot_process(S):
    n, _ = S.shape
    dates = np.linspace(0, n-1, n)

    plt.plot(dates, S)
    plt.show()

### test

In [5]:
# simulate_geometric_brownian_motion(
#     S0 = 100,
#     mu = 5/100,
#     sigma = 20/100,
#     n = 252, # 1 year
#     N = 100,
#     plot=True
# )

## European call option framework
> Black and Scholes (1973)
> Merton (1973)

$$ S_0 e^{-qT} \mathcal{N}(d_1) - K e^{-rT} \mathcal{N}(d_2)$$
where:
- $r$ is the risk free rate
- $q$ is the dividend yield
- $S_0$ the initial value of $S$
- $d_1 = \frac{ ln(S_0/K) + (r-q+\sigma^2/2)T}{\sigma \sqrt{T}}$
- $d_2 = d_1 - \sigma \sqrt{T}$


In [8]:
def black_scholes_d1(S, K, r, q, sigma, T):
  """
  Parameters
  ----------
  S: float
      Underlying price
  K: float
      Strike price
  r: float
      Risk-free rate
  q: float
      Dividend yield
  sigma: float
      Volatility
  T: int
      Residual maturity (in years)
  """
  return (np.log(S/K) + T*(r - q + 0.5*sigma**2) ) / (sigma * np.sqrt(T))

def black_scholes_call(S, K, r, q, sigma, T):
  """
  Parameters
  ----------
  S: float
      Underlying price
  K: float
      Strike price
  r: float
      Risk-free rate
  q: float
      Dividend yield
  sigma: float
      Volatility
  T: int
      Residual maturity
  """
  d1 = black_scholes_d1(S, K, r, q, sigma, T)
  d2 = d1 - sigma * np.sqrt(T)
  return S * np.exp(-q*T) * stats.norm.cdf(d1) - K * np.exp(-r*T) * stats.norm.cdf(d2)

def black_scholes_delta(S, K, r, q, sigma, T):
  """
  Parameters
  ----------
  S: float
      Underlying price
  K: float
      Strike price
  r: float
      Risk-free rate
  q: float
      Dividend yield
  sigma: float
      Volatility
  T: int
      Residual maturity
  """
  d1 = black_scholes_d1(S, K, r, q, sigma, T)
  return np.exp(-q*T) * stats.norm.cdf(d1)

### test

In [7]:
# print(black_scholes_call(100, 100, 2/100, 0, 20/100, 1) * 10000) # approx 90000 as per Cao & al (2019)
# print('\tvs 90000')
# print(black_scholes_delta(100, 100, 2/100, 0, 20/100, 1) * 10000) # approx 5800 as per Cao & al (2019)
# print('\tvs5800')

# print(black_scholes_delta(115, 100, 2/100, 0, 20/100, 6/12)) # approx 0.87 as per Cao & al (2019)
# print('\tvs 0.87')

# print(black_scholes_delta(115*(1.1), 100, 2/100, 0, 20/100, 5/12)) # approx 0.94 as per Cao & al (2019)
# print('\tvs 0.94')
# print(black_scholes_delta(115*(0.9), 100, 2/100, 0, 20/100, 5/12)) # approx 0.65 as per Cao & al (2019)
# print('\tvs 0.65')

## Financial data

In [9]:
def get_financial_data(underlying_params, option_params, trading_params):
    """Simulation of financial data as per modeling of the situation

    Parameters
    ----------
    underlying_params: tuple(float, float, float, float, int, int)
        Parameters for geometric Brownian simulation
        (initial underlying price, expected return, volatility, number of increment, number of trajectory)
        Here the number of increment corresponds to the year base (ex: 252 days per year)
    option_params:
        Parameters for Black and Scholes european option evaluation
        (spot price, strike, risk-free rate, dividend yield, volatility, residual maturity)
    trading_params: tuple(float, float)
        Trading parameters corresponding
        (trade frequency per day, trade cost)
    """

    S0, mu, sigma, n, N = underlying_params
    S, K, r, q, sigma, T = option_params
    trading_freq, trading_cost = trading_params

    # Cf 'Modeling of the situation' section (we only model step with rebalancing)
    rebalancing_n = int(T/trading_freq)
    rebalancing_interval = 1/n*trading_freq
    # We have indeed: rebalancing_n * rebalancing = T/n = option_life in annual base n

    # Spot prices
    S = simulate_geometric_brownian_motion(S0, mu, sigma, n=rebalancing_n, N=N, dt=rebalancing_interval)
    underlying_prices = S.T

    # Residual maturities, option_prices, option_deltas
    M_res = np.arange(T, -trading_freq, -trading_freq)
    res_maturities = M_res[M_res >= 0]
    option_prices = black_scholes_call(underlying_prices, K, r, q, sigma, res_maturities/n)
    option_deltas = black_scholes_delta(underlying_prices, K, r, q, sigma, res_maturities/n)

    return underlying_prices, option_prices, option_deltas, res_maturities

### test

In [9]:
# S0 = 100
# mu = 5/100
# sigma = 20/100
# n = 252 # will correspond to the year base in days in the function
# N = 100
# underlying_params = (S0, mu, sigma, n, N)

# S = 100
# K = 100
# r = 0
# q = 0
# T = 21
# option_params = (S, K, r, q, sigma, T)

# trading_freq = 4
# trading_cost = 1/100
# trading_params = (trading_freq, trading_cost)


# underlying_prices, option_prices, option_deltas, res_maturities = get_financial_data(underlying_params, option_params, trading_params)
# print(underlying_prices.shape)
# print(option_prices.shape)
# print(option_deltas.shape)

# # Plot
# fig, axes = plt.subplots(1, 3, figsize=(10,4))

# axes[0].plot(underlying_prices.T)
# axes[0].set_title('Underlying Prices')
# axes[0].set_xlabel('Days')
# axes[0].set_ylabel('Underlying Price')

# axes[1].plot(option_prices.T)
# axes[1].set_title('Option Prices')
# axes[1].set_xlabel('Days')
# axes[1].set_ylabel('Option Price')

# axes[2].plot(option_deltas.T)
# axes[2].set_title('Option Deltas')
# axes[2].set_xlabel('Days')
# axes[2].set_ylabel('Option Delta')

# plt.show()

# plt.plot(res_maturities.reshape(len(res_maturities),1))
# plt.xlabel('Steps')
# plt.ylabel('Days')
# plt.title('Residual Maturity')

## Environment

We suppose rebalancing of the position is done at time intervals $\Delta t$.
Rebalancing of the position is subject to trading costs $\kappa$.
The life of the option is $n \cdot \Delta t$.


**State**
At time $i \cdot \Delta t$.
Defined by 3 parameters:

- Asset price at time $i \Delta t$

- Holding of the asset from time $(i-1) \cdot \Delta t$ to time $i \cdot \Delta t$

- Time to maturity



**Action**
At time $i \cdot \Delta t$.
Amount of asset to be held from $i \cdot \Delta t$ to $(i+1) \cdot \Delta t$.



**Reward - Accounting P&L Formulation**
Reward corresponds to .

$$ R_0 = V_0 -\kappa \cdot |S_0 \cdot H_0| $$
$$ R_{i+1} = V_{i+1} - V_i + H_i \cdot (S_{i+1}-S_i) - \kappa \cdot |{S_{i+1} \cdot (H_{i+1}-H_i)}| $$
$$ R_n = V_{n} - V_{n-1} + H_{n-1} \cdot (S_{n}-S_{n-1}) -\kappa \cdot |S_{n} \cdot H_{n}| $$
for $0 \le i \le n$
where:
- $S_i$ is the asset price at the beginning of the period
- $H_i$ is the holding between time $i \Delta t$ and $(i+1) \Delta t$
- $\kappa$ is the trading cost as a proportion of value
- $V_i$ is the value of the derivative position at the beginning of period $i$
- $V_n$ is the derivative position payoff

Note that $R_0$ is negative (short call position value + hedge set up).


In [10]:
class HedgingEnv(gym.Env): # https://gymnasium.farama.org/api/env/
    """An environment for heding simulations

    Based on gymnasium environments
    Action space is defined in classes inheriting from HedgingEnv.
    Reward corresponds to wealth.

    Attributes
    ----------
    self.trading_freq
    self.trading_cost
    self.strike
    self.res_maturities
    self.underlying_prices
    self.options_prices
    self.option_deltas
    self.n_trajectory
    self.n_step
    self.index_trajectory
    self.index_step
    self.state_dim
    self.state
    self.reward
    self.done
    self.info

    Methods
    -------
    __init__(self)
    reset(self)
    step(self, action)
    plot(self)
    """

    def __init__(
        self,
        underlying_params = (100, 5/100, 20/100, 252, 50000),
        option_params = (100, 100, 0, 0, 20/100, 21),
        trading_params = (1, 1/100)
        ):

        # Parameters
        _, self.strike, _, _, _, _ = option_params
        self.trading_freq, self.trading_cost = trading_params

        # Observation Space
        underlying_prices, option_prices, option_deltas, res_maturities = get_financial_data(
            underlying_params, option_params, trading_params
        )

        self.underlying_prices = underlying_prices
        self.option_prices = option_prices
        self.option_deltas = option_deltas
        self.res_maturities = res_maturities

        self.n_trajectory = underlying_prices.shape[0]
        self.n_step = underlying_prices.shape[1]

        # State
        self.index_trajectory =  -1
        self.index_step = -1

        self.state_dim = 3
        self.state = np.array([])

        # Reward
        self.reward = 0
        self.rewards = []

        # Done
        self.done = False

        # Info
        self.info = [self.index_trajectory,self.index_step]


    def reset(self):
        """Resets the environments.

        Here resetting corresponds to a change of trajectory.
        """

        # Change trajectory
        self.index_trajectory = (self.index_trajectory + 1) % self.n_trajectory
        self.index_step = 0

        # Reset state
        self.state = np.array([
            self.underlying_prices[self.index_trajectory, self.index_step],
            self.index_step,
            self.res_maturities[self.index_step]
        ])

        # Reward (setting up the hedge at t=0)
        underlying_price = self.underlying_prices[self.index_trajectory,self.index_step]
        option_price = self.option_prices[self.index_trajectory,self.index_step]
        option_delta = self.option_deltas[self.index_trajectory,self.index_step]

        self.reward = \
            option_price \
            - self.trading_cost * np.abs(underlying_price * option_delta)
        self.rewards = [self.reward]

        # Done
        self.done = False

        # Info
        self.info = [self.index_trajectory, self.index_step]

        return self.state


    def step(self, action):

        # State
        underlying_price = self.state[0]
        position = self.state[1]
        option_price = self.option_prices[self.index_trajectory, self.index_step]

        # New state
        new_index_step = self.index_step + 1
        new_underlying_price = self.underlying_prices[self.index_trajectory, new_index_step]
        new_position = action
        new_option_price = self.option_prices[self.index_trajectory, new_index_step]
        new_res_maturity = self.res_maturities[new_index_step]

        new_state = np.array([new_underlying_price, new_position, new_res_maturity])

        # Reward (liquidation of the hedge at t=n)
        if new_index_step == self.n_step - 1:
            new_option_price = np.max(underlying_price - self.strike, 0) # (St-K)+
            self.done = True
        else:
            new_option_price = self.option_prices[self.index_trajectory, new_index_step]

        new_reward = new_option_price - option_price\
            + position * (new_underlying_price - underlying_price)\
            - self.trading_cost * np.abs(new_underlying_price * (new_position - position))

        self.rewards.append(new_reward)

        # Updates
        self.index_step = new_index_step
        self.state = new_state
        self.reward = new_reward
        self.info = [self.index_trajectory,self.index_step]

        return self.state, self.reward, self.done, self.info


    def plot(self):
        """Plots current state"""

        if self.index_step == -1:
            plt.plot(self.underlying_prices.T)
            plt.title('Simulation')
            plt.xlabel('Steps')
            plt.ylabel('Underlying Price')
            plt.show()

        else:
            # Plot
            fig, axes = plt.subplots(1, 2, figsize=(10,4))

            axes[0].plot(self.underlying_prices[self.index_trajectory].T)
            axes[0].set_title(f"Trajectory {self.index_trajectory}")
            axes[0].set_xlabel('Step')
            axes[0].set_ylabel('Underlying Price')
            axes[0].axvline(self.index_step, c='red')
            axes[0].legend([f"Position: {self.state[1]}",
                            f"Step: {self.index_step}"])

            axes[1].plot(self.rewards)
            axes[1].set_title('Rewards')
            axes[1].set_xlabel('Step')
            axes[1].set_ylabel('Reward')
            axes[1].legend([f"Reward: {np.round(self.reward,2)}"])

            plt.show()


class ContinuousActionHedgingEnv(HedgingEnv):
    """Hedging env with continuous action space

    Attributes
    ----------
    self.action_space
    self.action_meaning
    self.action_dim

    Methods
    -------
    __init__(self)
    """

    def __init__(
        self,
        underlying_params = (100, 5/100, 20/100, 252, 50000),
        option_params = (100, 100, 0, 0, 20/100, 21),
        trading_params = (1, 1/100)
        ):

        super().__init__(underlying_params, option_params, trading_params)

        # Action space
        self.action_space = spaces.Box(low=np.array([0]), high=np.array([100]), dtype=np.float32)
        self.action_dim = 1


class DiscreteActionHedgingEnv(HedgingEnv):
    """Hedging env with discrete action space

    Attributes
    ----------
    self.action_space
    self.action_meaning
    self.action_dim

    Methods
    -------
    self.__init__()
    self.step()
    """

    def __init__(
        self,
        underlying_params = (100, 5/100, 20/100, 252, 50000),
        option_params = (100, 100, 0, 0, 20/100, 21),
        trading_params = (1, 1/100),
        action_list=[0,25,50,100]
        ):

        super().__init__(underlying_params, option_params, trading_params)

        # Action space
        self.action_dim = len(action_list)
        self.action_space = spaces.Discrete(self.action_dim)

        self.action_mapping = {}
        for index, value in enumerate(action_list):
            self.action_mapping[index] = value


    def step(self, action):
        if action in range(self.action_dim):
            return super().step(self.action_mapping[action])
        else:
            return super().step(action)

### instanciation

In [11]:
# S0 = 100
# mu = 5/100
# sigma = 20/100
# n = 252 # will correspond to the year base in days in the function
# N = 100
# underlying_params = (S0, mu, sigma, n, N)

# S = 100
# K = 100
# r = 0
# q = 0
# T = 21
# option_params = (S, K, r, q, sigma, T)

# trading_freq = 1
# trading_cost = 1/100
# trading_params = (trading_freq, trading_cost)

# env = DiscreteActionHedgingEnv(underlying_params, option_params, trading_params)

### explanations

At $t_0$:
- Spot
- Option value
- Initial position = Option Delta
> Initial wealth = Option value - Trading cost * Initial position

*Action (position for next period)*

At $t+1$:
- New spot
- New option price
- PNL (reward)

In [12]:
# env.reset()

# print(f"Spot: {env.underlying_prices[env.index_trajectory, env.index_step]}")
# print(f"Option: {env.option_prices[env.index_trajectory, env.index_step]}")
# print(f"Position: {env.option_deltas[env.index_trajectory, env.index_step]}")
# print(f"Wealth: {env.reward}")

# while not env.done:
#     print('\n')
#     action = env.option_deltas[env.index_trajectory, env.index_step]
#     step, reward, done, info = env.step(action)
#     print(f"ACTION (position for next period): {action}")
#     print('\n')

#     print(f"Spot: {env.underlying_prices[env.index_trajectory, env.index_step]}")
#     print(f"Option: {env.option_prices[env.index_trajectory, env.index_step]}")
#     print(f"Wealth: {env.reward}")


### test

In [13]:
# # test global
# env = DiscreteActionHedgingEnv(underlying_params, option_params, trading_params)

# env.reset()
# steps = []
# rewards = []

# for i in range(0,int(T/trading_freq)):

#     action = env.action_space.sample().item()
#     steps.append(env.step(action))
#     rewards.append(env.reward)

#     if i == 10:
#         env.plot()

# Discrete action space - DQN
References:
- 2013, Mnih & al - Playing Atari with Reinforcement Learning
- https://github.com/algila/Deep-Q-Learning-DQN
- G.Turinici notebooks inspired from the above
- https://github.com/tdmdal/rl-hedge-2019/tree/master


**Action space**
Action are in ${25,50,75}$


## Constants

In [11]:
# Paremeters as per Table 1 in the article
TRADING_FREQ = 1
TRADING_COST = 1/100
TRADING_PARAMS = (TRADING_FREQ,
                  TRADING_COST)

# Parameters for underlying simulation
UNDERLYING_INITIAL_PRICE = 100
EXPECTED_RETURN = 5/100
VOLATILITY = 20/100
YEAR_BASE = 252
TRAJECTORY_N = 50000
UNDERLYING_PARAMS = (UNDERLYING_INITIAL_PRICE,
                     EXPECTED_RETURN,
                     VOLATILITY,
                     YEAR_BASE,
                     TRAJECTORY_N)

# Paremeters for option computation
STRIKE = 100
RISK_FREE_RATE = 0
DIVIDEND_YIELD = 0
MATURITY = 21
OPTION_PARAMS = (UNDERLYING_INITIAL_PRICE,
                 STRIKE,
                 RISK_FREE_RATE,
                 DIVIDEND_YIELD,
                 VOLATILITY,
                 MATURITY)

# Parameters for Replay Memory
REPLAY_MEMORY_SIZE = 6000 # 600000
MINIBATCH_SIZE = 64 # 128

# Parameters for DQN
DISCOUNT_FACTOR = 0.99
LEARNING_RATE = 1e-4

# Parameters for main loop
epsilon = 1.0
EPSILON_DECAY = 0.99994
EPSILON_MIN = 0.01
EPSILON_DECAY_START = 25 # 2500
epsilon_params = (epsilon,
                  EPSILON_DECAY,
                  EPSILON_MIN,
                  EPSILON_DECAY_START)

EPISODE_N = 500 # 50000
EPISODE_N_TEST = 1000 # 10000
TARGET_MODEL_UPDATE_FREQUENCY = 25 # 2500
STATS_FREQUENCY = 10 # 1000

MODEL_NAME = 'DQN - small run'
WEIGHT_SAVE_FREQUENCY = 10 # 1000

# Evaluation parameters
RISK_ADVERSE_CONSTANT = 1.5
EVALUATION_PARAMETERS = (RISK_ADVERSE_CONSTANT)

## Replay Memory
$$ e_t = (s_t, a_t, r_{t+1}, s_{t+1}) $$

> https://github.com/algila/Deep-Q-Learning-DQN/blob/main/DQN_paper_2013.py
> https://github.com/fg91/Deep-Q-Learning/blob/master/DQN.ipynb

In [12]:
class ReplayMemory(object):
    """Replay Memory that stores the last "size" experiences

    Attributes
    ----------
    self.buffer: List
    self.size: int
        Number of stored experiences
    self.batch_size: int
        Number of experiences returned in a minibatch
    self.experience_index: int

    Methods
    -------
    self.__init__
    self.__len__
    self.add
    self.get
    self.sample
    """

    def __init__(
        self,
        size = 600000,
        batch_size = 128
        ):
        """
        Parameters
        ----------
        size: int
            Number of stored experiences
        batch_size: int
            Number of experiences returned in a minibatch
        """

        self.buffer = []
        self.size = size
        self.batch_size = batch_size
        self.new_experience_index = 0


    def __len__(self):
        return len(self.buffer)


    def add(self, state, action, new_reward, new_state, done):
        """Adds an experiences to the buffer"""

        new_experience = (state, action, new_reward, new_state, done)

        if self.new_experience_index >= self.__len__():
            self.buffer.append(new_experience)
        else:
            self.buffer[self.new_experience_index] = new_experience

        self.new_experience_index = (self.new_experience_index + 1) % self.size


    def get(self, index):
        """Gets an experience from the buffer"""

        if self.__len__()==0:
            raise ValueError("Replay memory is empty!")

        try:
           return self.buffer[index]
        except:
          print('No such index in Replay Memory')
          return None


    def sample(self):
        """Sample a batch of experiences of size ```batch_size``` from ```buffer```."""

        # Check if enough experiences
        if self.__len__() < self.batch_size:
            raise ValueError('Not enough experiences in buffer to get a minibatch')

        # Get random indexes
        indexes = [random.randint(0, self.__len__() - 1) for _ in range(self.batch_size)]

        # Get corresponding experiences
        state_list, action_list, new_reward_list, new_state_list, done_list = [], [], [], [], []

        for i in indexes:

            experience = self.buffer[i]
            state, action, new_reward, new_state, done = experience

            state_list.append(np.array(state, copy=False))
            action_list.append(action)
            new_reward_list.append(new_reward)
            new_state_list.append(np.array(new_state, copy=False))
            done_list.append(done)

        return np.array(state_list), np.array(action_list), np.array(new_reward_list), np.array(new_state_list), np.array(done_list)


### test

In [16]:
# replay_memory = ReplayMemory(size=REPLAY_MEMORY_SIZE, batch_size=MINIBATCH_SIZE)

# env = DiscreteActionHedgingEnv(underlying_params, option_params, trading_params)

# env.reset()
# state = env.state

# for j in tqdm(range(0,10000)):
#     for i in range(0,int(T/trading_freq)):
#         action = env.action_space.sample().item()
#         new_state, new_reward, done, info = env.step(action)
#         replay_memory.add(state, action, new_reward, new_state, done)
#         state = new_state
#     env.reset()

# print(len(replay_memory))

# a, b, c, d, e = replay_memory.sample()
# print(a.shape)

In [145]:
# replay_memory = ReplayMemory(
#     size=REPLAY_MEMORY_SIZE,
#     batch_size=MINIBATCH_SIZE
# )

# env = ContinuousActionHedgingEnv()

# env.reset()
# state = env.state

# for j in tqdm(range(0,10000)):
#     for i in range(0,21):
#         action = env.action_space.sample().item()
#         new_state, new_reward, done, info = env.step(action)
#         replay_memory.add(state, action, new_reward, new_state, done)
#         state = new_state
#     env.reset()

# print(len(replay_memory))

# a, b, c, d, e = replay_memory.sample()
# print(a.shape)

## DQN Agent

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

class DQNAgent():
    """Deep Q-Network agent

    Attributes
    ----------
    self.env
    self.state_dim
    self.action_dim
    self.discount_factor
    self.learning_rate
    self.model
    self.target_model
    self.target_update_counter

    Methods
    -------
    self._init_
    self.create_model
    self.describe_model
    self.train
    self.update_target_model
    self.get_q
    """

    def __init__(
        self,
        env,
        learning_rate,
        discount_factor
        ):

        # Dimensions (for neural network)
        self.state_dim = env.state_dim
        self.action_dim = env.action_dim

        # Parameters
        self.discount_factor = discount_factor
        self.learning_rate = learning_rate

        # Main model
        self.model = self.create_model()

        # Target network
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        # Used to count when to update target network with main network's weights
        self.target_update_counter = 0


    def create_model(self):
        """Creates Deep Learning model

        Hidden layers activation:
        - relu
        Output layer activation:
        - sigmoid (for binary classification)
        - softmax (for multi-class classification)
        - linear (for regression problems with continuous values to be predicted)

        Loss:
        - mse (for regression tasks)
        - categorical_crossentropy (for multi-class classification)
        """

        model = Sequential() # weights are randomly initialized when new instance is created

        model.add(Dense(32, input_dim=self.state_dim))
        model.add(Activation('relu'))

        model.add(Dense(64))
        model.add(Activation('relu'))

        model.add(Dense(self.action_dim))
        model.add(Activation('softmax'))

        model.compile(optimizer=Adam(self.learning_rate), loss='categorical_crossentropy')

        # optimizer automatically computes the gradient and update model weights

        return model


    def describe_model(self):
        print(self.model.summary())


    def train(self, minibatch):

        # Read minibatch
        state_list, action_list, new_reward_list, new_state_list, done_list = minibatch
        minibatch_size = len(state_list)

        # Compute Q_true(St+1) et Q_ann(St)
        Q_current_states = self.model.predict(state_list, verbose=0)
        Q_future_states = self.target_model.predict(new_state_list, verbose=0)

        X = []
        Y = []

        # Enumerate batches
        for step in range(MINIBATCH_SIZE):
            state = state_list[step]
            action = action_list[step]
            new_reward = new_reward_list[step]
            new_state = new_state_list[step]
            done = done_list[step]

            # Bellman equation.  Q = reward + gamma*max Q',

            # Update Q-target
            if not done:
                new_Q_value = new_reward + self.discount_factor * np.max(Q_future_states[step])
            else:
                new_Q_value = new_reward

            # Update estimated Q-values
            Q_current_states[step][action] = new_Q_value

            # And append to our training data
            X.append(state)
            Y.append(Q_current_states[step])

        # Fit on all samples as one batch, NO log file saved. Quicker simulation
        # Fit computes propagation, loss, backpropagation and weight update
        self.model.fit(np.array(X), np.array(Y),
                       batch_size=minibatch_size,
                       verbose=1,
                       epochs=1,
                       shuffle=False)


    # update target model function frames as verified into main (9*)
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        self.target_update_counter = 0

    def get_q(self, state):
      return self.model.predict(np.array(state).reshape(1,-1))


### test

In [18]:
# env = DiscreteActionHedgingEnv(UNDERLYING_PARAMS, OPTION_PARAMS, TRADING_PARAMS)

# agent = DQNAgent(
#   env=env,
#   learning_rate=LEARNING_RATE,
#   discount_factor=DISCOUNT_FACTOR
# )

# agent.describe_model()

# agent.train(replay_memory.sample())

# state = (110, 25, 10)
# agent.get_q(state)

## Algorithm

> Mnih and al (2013)

In [14]:
from tqdm import tqdm
import os

class MainDQN():
    """This class represents Mnih & al (2013) algorithm"""

    def __init__(
        self,
        underlying_params = (100, 5/100, 20/100, 252, 50000),
        option_params = (100, 100, 0, 0, 20/100, 21),
        trading_params = (1, 1/100),
        replay_memory_size = 600000,
        minibatch_size = 128,
        learning_rate = 1e-4,
        discount_factor = 0.99,
        epsilon_params = (1.0, 0.9994, 0.01, 2500),
        episode_n = 50000,
        model_name = 'DQN',
        stats_frequency = 1000,
        weight_save_frequency = 1000,
        target_model_update_frequency = 2500,
        load_flag = False,
        tag = ''
        ):

        # Environment
        self.env = DiscreteActionHedgingEnv(
            underlying_params,
            option_params,
            trading_params
        )

        # Replay Memory
        self.replay_memory = ReplayMemory(
            replay_memory_size,
            minibatch_size
        )

        # Network
        self.agent = DQNAgent(
            self.env,
            learning_rate ,
            discount_factor
        )
        self.model_name = model_name
        self.tag = tag
        self.load_flag = load_flag
        if load_flag:
            self.load_weights()

        # Algo
        self.epsilon, self.epsilon_decay, self.epsilon_min, self.epsilon_decay_start = epsilon_params

        self.episode_n = episode_n
        self.episode_count = 0
        self.step_count = 0

        self.stats_frequency = stats_frequency
        self.episode_final_reward_list = []
        self.stats = {}

        self.target_model_update_frequency = target_model_update_frequency
        self.weight_save_frequency = weight_save_frequency


    def load_weights(self):
        """Loads weights for model"""

        if self.tag == "":
            file = f"Cao and al, 2019 - models/{self.model_name}.h5"
        else:
            file = f"Cao and al, 2019 - models/{self.model_name} - {self.tag}.h5"

        if os.path.exists(file):
            self.agent.model.load_weights(file)
            self.agent.target_model.load_weights(file)


    def epsilon_greedy_action_discrete_case(self, state):
        if np.random.rand() <= self.epsilon:
            action = self.env.action_space.sample().item()
        else:
            action = np.argmax(self.agent.get_q(state)) # [0]
        return action


    def epsilon_update(self):
        if self.epsilon >= self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    def get_stats(self):
        if not self.episode_final_reward_list:
            print('No stats for empty reward list.')
        else:
            self.stats['episode'] = self.episode_count
            self.stats['average'] = np.mean(self.episode_final_reward_list)
            self.stats['min'] = np.min(self.episode_final_reward_list)
            self.stats['max'] = np.max(self.episode_final_reward_list)
            self.stats['last'] = self.episode_final_reward_list[-1]
            print(f"Episode: {self.stats['episode']}\nAverage Final Wealth: {self.stats['average']}\nMin Final Wealth: {self.stats['min']}\nMax Final Wealth: {self.stats['max']}\nLast Final Wealth: {self.stats['last']}")


    def train(self):

        if self.load_flag:
            start = self.tag
        else:
            start = 1

        for episode in tqdm(range(start, self.episode_n + 1)): # 1 episode = 1 trajectory

            self.episode_count += 1

            # 3.
            # Initialize sequence
            state = self.env.reset()
            episode_reward_list = []

            while not self.env.done: # done when meeting end of trajectory

                self.step_count += 1

                # 4.
                # Select epsilon-greedy action
                action = self.epsilon_greedy_action_discrete_case(state)

                # 5. 6.
                # Execute action
                new_state, new_reward, done, info = self.env.step(action)

                # 7.
                # Store transition in replay memory
                self.replay_memory.add(state, action, new_reward, new_state, done)

                # 8.
                # Sample random minibatch
                if  len(self.replay_memory) > self.replay_memory.batch_size:
                    minibatch = self.replay_memory.sample()
                    self.agent.train(minibatch)
                    del minibatch

                # 9.
                # Reset Q_hat every C steps
                if  self.step_count % self.target_model_update_frequency == 0 and self.step_count > self.replay_memory.batch_size:
                    self.agent.update_target_model()

                # Close loop
                state = new_state
                episode_reward_list.append(new_reward)


            # Decay epsilon. Only start after replay memory is over min size
            if self.step_count > self.epsilon_decay_start:
                self.epsilon_update()

            # Follow-up
            if episode % self.stats_frequency == 0 and episode > 1:
                self.episode_final_reward_list.append(np.sum(episode_reward_list))
                self.get_stats()

            # Save weights
            if episode % self.weight_save_frequency == 0:
                self.agent.model.save(f"Cao and al, 2019 - models/{self.model_name} - {episode}.h5")


        self.agent.model.save(f"Cao and al, 2019 - models/{self.model_name}.h5")


    def test(
        self,
        delta_action_flag = False,
        episode_n = 10000,
        risk_adverse_constant = 1.5,
        verbose_frequency = 1000
        ):
        """Test should be run in another instance"""

        # Reinitialize
        self.epsilon = -1
        episode_index = 0

        for episode in tqdm(range(1, episode_n + 1)): # 1 episode = 1 trajectory

            episode_index += 1

            # Initialize
            state = self.env.reset()
            episode_action_list = []
            episode_reward_list = []

            while not self.env.done:

                # Select action
                if delta_action_flag:
                    action = self.env.option_deltas[self.env.index_trajectory,self.env.index_step] * 100
                else:
                    action = self.epsilon_greedy_action_discrete_case(state)

                # Take action
                new_state, new_reward, done, info = self.env.step(action)

                # Close loop
                episode_action_list.append(action)
                episode_reward_list.append(new_reward)
                state = new_state

            # Follow-up
            self.episode_final_reward_list.append(np.sum(episode_reward_list))

            if episode % verbose_frequency == 0:
                R_final_mean = np.mean(self.episode_final_reward_list)
                R_final_var = np.var(self.episode_final_reward_list)
                Y = -R_final_mean + risk_adverse_constant*np.sqrt(R_final_var)

                print(f"\nEpisode {episode} final reward is: {self.episode_final_reward_list[-1]}")
                print(f"Final reward mean: {R_final_mean}")
                print(f"Final reward var: {R_final_var}")
                print(f"Y = {Y}")



### small run - train

In [ ]:
# Parameters 'small run'
# algo = MainDQN(
#     underlying_params = (100, 5/100, 20/100, 252, 50000),
#     option_params = (100, 100, 0, 0, 20/100, 21),
#     trading_params = (1, 1/100),
#     replay_memory_size = 6000,
#     minibatch_size = 64,
#     learning_rate = 1e-4,
#     discount_factor = 0.99,
#     epsilon_params =(1.0, 0.99994, 0.01, 25),
#     episode_n = 500,
#     model_name = 'DQN - small run',
#     stats_frequency = 10,
#     weight_save_frequency = 10,
#     target_model_update_frequency = 25
# )

# algo.train()

### small run - test

In [ ]:
# algo = MainDQN(
#     underlying_params = (100, 5/100, 20/100, 252, 50000),
#     option_params = (100, 100, 0, 0, 20/100, 21),
#     trading_params = (1, 1/100),
#     replay_memory_size = 6000,
#     minibatch_size = 64,
#     learning_rate = 1e-4,
#     discount_factor = 0.99,
#     epsilon_params =(1.0, 0.99994, 0.01, 25),
#     episode_n = 500,
#     model_name = 'DQN - small run',
#     stats_frequency = 10,
#     weight_save_frequency = 10,
#     target_model_update_frequency = 25,
#     load_flag = True,
#     tag = '' # no tag means final weights
# )

# algo.test(
#     # delta_action_flag = False,
#     delta_action_flag = True,
#     episode_n = 100,
#     risk_adverse_constant = 1.5,
#     verbose_frequency = 10
# )

# DQN results (delta_action_flag = False)
# Episode 100 final reward is: 318.0843837821259
# Final reward mean: -95.04987783477348
# Final reward var: 294240.9806954759
# Y = 908.7096387697854

# Delta hedging (delta_action_flag = True)
# Episode 100 final reward is: -373.637076533253
# Final reward mean: -160.5699514605786
# Final reward var: 155184.62675484293
# Y = 751.4721516582057

### full run

In [ ]:
# algo_train = MainDQN()

# algo_test = MainDQN(load_flag = True)
# algo_test.test(delta_action_flag = False)
# algo_test.test(delta_action_flag = True)

# Continuous action space - Actor Critic

References:
> https://www.tensorflow.org/tutorials/reinforcement_learning/actor_critic?hl=fr



**Greedy action**
The greedy action $a$ is the one that minimizes:
$$ F(S_t,a) = Q_1(S_t,a) + c \cdot \sqrt{ Q_2(S_t,a) - {Q_1(S_t,a)}^2} $$

**Q-values**
$Q_1(S_t, A_t; \omega_1)$
estimates the expected cost for state-action combination.
$Q_2(S_t, A_t; \omega_2)$ estimates the expected *square* cost for state-action combination.



**Loss functions**

$ L_1 = (R_{t+1} + \gamma \cdot Q_1(S_{t+1}, \pi (S_{t+1})) - Q_1(S_t, A_t; \omega_1))^2 $

$ L_2 = (R^2_{t+1} + \gamma^2 \cdot Q_2(S_{t+1}, \pi (S_{t+1})) + 2 \cdot \gamma \cdot R_{t+1} \cdot Q_1(S_{t+1},\pi (S_{t+1})) - Q_2(S_t, A_t; \omega_2))^2 $

**Policy function**

$\pi (S_t;\theta)$ is updated with:

$$ \theta ← \theta - \alpha \cdot \nabla_{\theta} F (S_t, \pi(S_t;\theta)) $$







## Continuous Env

### test

In [ ]:
# env = ContinuousActionHedgingEnv(
#     underlying_params = (100, 5/100, 20/100, 252, 100),
#     option_params = (100, 100, 0, 0, 20/100, 21),
#     trading_params = (1, 1/100)
#     )

# env.reset()

# for i in range(0,20):

#     print(f"\nInfo: {env.info}")
#     action = env.action_space.sample().item()
#     print(f"Action: {action}")
#     step = env.step(action)
#     print(f"State: {step}")
#     reward = env.reward
#     print(reward)

## Actor Critic

In [30]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Lambda, concatenate
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K


class ActorCritic():
    """Deep Q-Network agent

    Attributes
    ----------
    self.env
    self.state_dim
    self.action_dim
    self.discount_factor
    self.learning_rate
    self.model
    self.target_model
    self.target_update_counter

    Methods
    -------
    self._init_
    self.create_model
    self.describe_model
    self.train
    self.update_target_model
    self.get_q
    """

    def __init__(
        self,
        env,
        learning_rate = 1e-4,
        discount_factor = 0.99,
        risk_adverse_constant = 1.5
        ):

        # Parameters
        self.discount_factor = discount_factor
        self.learning_rate = learning_rate
        self.risk_adverse_constant = risk_adverse_constant

        # Dimensions (for neural network)
        self.state_dim = env.state_dim
        self.action_dim = env.action_dim

        # Networks
        self.actor = self.create_actor()
        self.Q1, self.Q2, self.F = self.create_critic()

        # Target networks
        self.actor_target = self.create_actor()
        self.actor_target.set_weights(self.actor.get_weights())

        self.Q1_target, self.Q2_target, self.F_target = self.create_critic()
        self.Q1_target.set_weights(self.Q1.get_weights())
        self.Q2_target.set_weights(self.Q2.get_weights())

        # Used to count when to update target network with main network's weights
        self.target_update_counter = 0


    def create_actor(self):
        """Creates Deep Learning model

        Hidden layers activation:
        - relu
        Output layer activation:
        - sigmoid (for binary classification)
        - softmax (for multi-class classification)
        - linear (for regression problems with continuous values to be predicted)

        Loss:
        - mse (for regression tasks)
        - categorical_crossentropy (for multi-class classification)
        """

        model = Sequential()
        model.add(Input(self.state_dim))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1, activation='linear'))
        model.compile(optimizer=Adam(self.learning_rate), loss='mse')

        return model


    def create_critic(self):
        """Creates Deep Learning model

        Hidden layers activation:
        - relu
        Output layer activation:
        - sigmoid (for binary classification)
        - softmax (for multi-class classification)
        - linear (for regression problems with continuous values to be predicted)

        Loss:
        - mse (for regression tasks)
        - categorical_crossentropy (for multi-class classification)
        """

        state_input = Input(shape=(self.state_dim,))
        action_input = Input(shape=(1,))

        # Q1 will compute the expected cost of an action
        Q1_input = concatenate([state_input, action_input])
        Q1_hidden_layer_1 = Dense(32, activation='relu')(Q1_input)
        Q1_hidden_layer_2 = Dense(64, activation='relu')(Q1_hidden_layer_1)
        Q1_output_layer = Dense(1, activation='linear')(Q1_hidden_layer_2)

        Q1 = Model(inputs = [state_input,action_input], outputs = Q1_output_layer)
        Q1.compile(optimizer=Adam(self.learning_rate), loss='mse')

        # Q2 will compute the square expected cost of an action
        Q2_input = concatenate([state_input, action_input])
        Q2_hidden_layer_1 = Dense(32, activation='relu')(Q2_input)
        Q2_hidden_layer_2 = Dense(64, activation='relu')(Q2_hidden_layer_1)
        Q2_output_layer = Dense(1, activation='linear')(Q2_hidden_layer_2)

        Q2 = Model(inputs = [state_input,action_input], outputs = Q2_output_layer)
        Q2.compile(optimizer=Adam(self.learning_rate), loss='mse')

        # F (as refered to in the article)
        # We build a model for F as F is function of Q1 and Q2
        # and we will need to compute F gradient
        F = Model(
            inputs = [state_input, action_input],
            outputs = Lambda(lambda Q: Q[0] - self.risk_adverse_constant * K.sqrt(K.max(Q[1] - Q[0]**2, 0)))\
                ([Q1_output_layer, Q2_output_layer])
        )
        F.compile(optimizer=Adam(self.learning_rate), loss='mse')

        return Q1, Q2, F


    def describe_model(self):
        print('\nActor')
        print(self.actor.summary())
        print('\nCritic Q1')
        print(self.Q1.summary())
        print('\nCritic Q2')
        print(self.Q2.summary())
        print('\nF')
        print(self.F.summary())


    def train(self, minibatch):

        # Read minibatch
        state_array, action_array, new_reward_array, new_state_array, done_array = minibatch
        minibatch_size = len(state_array)

        # Get next action
        next_action_array = self.actor_target.predict(new_state_array)

        # Compute target Q-values
        Q1_next_state = self.Q1_target.predict([new_state_array, next_action_array])
        Q2_next_state = self.Q2_target.predict([new_state_array, next_action_array])

        y_Q1 = new_reward_array \
            + (1-done_array) * self.discount_factor * Q1_next_state
        y_Q2 = new_reward_array**2 \
            + (1-done_array) * self.discount_factor**2 * Q2_next_state \
            + (1-done_array) * 2 * self.discount_factor * new_reward_array * Q1_next_state

        # Gradient_descent
        # Note that by construction, F is fitted whenever Q1 or Q2 are fitted
        self.Q1.fit(
            [state_array, action_array],
            y_Q1,
            batch_size=minibatch_size,
            verbose=0,
            epochs=1,
            shuffle=False
        )

        self.Q2.fit(
            [state_array, action_array],
            y_Q2,
            batch_size=minibatch_size,
            verbose=0,
            epochs=1,
            shuffle=False
        )

        self.update_actor(state_array)


    def update_actor(self, state_array):

        with tf.GradientTape() as tape:
            action_array = self.actor(state_array)
            F_output = self.F([state_array, action_array])

        actor_gradients = tape.gradient(F_output, self.actor.trainable_variables)

        actor_optimizer = tf.keras.optimizers.Adam(self.learning_rate)
        actor_optimizer.apply_gradients(zip(actor_gradients, self.actor.trainable_variables))


        # Method 1: Rebuild crtic gradient - NOT AVAILABLE IN EAGER MODE
        # action_array = self.actor(state_array)
        # F_input = self.F.input
        # F_output = self.F.output
        # F_gradient = K.gradients(F_input, F_output[1])
        # F_gradient_function = K.function([F_input[0], F_input[1]], F_gradient)

        # actor_gradient = np.array(F_gradient_function([state_array, action_array]))[0]

        # actor_optimizer = tf.keras.optimizers.Adam(self.learning_rate)
        # actor_optimizer.apply_gradients(zip(actor_gradient, self.actor.trainable_variables))


        # Method 2 - Track operations
        # action_array = self.actor(state_array)
        # with tf.GradientTape() as tape:
        #     actor_gradient = tape.gradient(
        #         self.F([state_array,action_array]),
        #         self.actor.trainable_variables
        #     )

        # actor_optimizer = tf.keras.optimizers.Adam(self.learning_rate)
        # actor_optimizer.apply_gradients(zip(actor_gradient, self.actor.trainable_variables))


    def update_target_model(self):
        self.Q1_target.set_weights(self.Q1.get_weights())
        self.Q2_target.set_weights(self.Q2.get_weights())
        self.actor_target.set_weights(self.actor.get_weights())


    def save_weights(self, model_name, tag=None):
        """Saves model weights"""

        if tag:
            folder_path = f"Cao and al, 2019 - models/{model_name} - {tag}/"
        else:
            folder_path = f"Cao and al, 2019 - models/{model_name}/"

        os.makedirs(folder_path, exist_ok=True)

        self.actor.save_weights(folder_path + "Actor.h5")
        self.Q1.save_weights(folder_path + "Q1.h5")
        self.Q2.save_weights(folder_path + "Q2.h5")


    def load_weights(self, model_name, tag = None):
        """Loads model weights"""

        if tag:
            folder_path = f"Cao and al, 2019 - models/{model_name} - {tag}/"
        else:
            folder_path = f"Cao and al, 2019 - models/{model_name}/"

        self.actor.load_weights(folder_path + "Actor.h5")
        self.Q1.load_weights(folder_path + "Q1.h5")
        self.Q2.load_weights(folder_path + "Q2.h5")

### test

In [12]:
# # FILL REPLAY MEMORY
# env = ContinuousActionHedgingEnv()
# replay_memory = ReplayMemory(
#     size=6000,
#     batch_size=64
# )
# env.reset()
# state = env.state
# for j in tqdm(range(0,10000)):
#     for i in range(0,21):
#         action = env.action_space.sample().item()
#         new_state, new_reward, done, info = env.step(action)
#         replay_memory.add(state, action, new_reward, new_state, done)
#         state = new_state
#     env.reset()

In [13]:
# ac = ActorCritic(env)
# ac.describe_model()
# ac.train(replay_memory.sample())

# state = tf.expand_dims((110, 25, 10), axis=0)
# ac.save_weights('ACtest')
# print(ac.actor.predict(state)[0][0])

# ac_test = ActorCritic(env)
# ac_test.load_weights('ACtest')
# print(ac_test.actor.predict(state)[0][0])

## Algorithm

In [33]:
from tqdm import tqdm
import os

class MainAC():
    """This class represents Mnih & al (2013) algorithm"""

    def __init__(
        self,
        underlying_params = (100, 5/100, 20/100, 252, 50000),
        option_params = (100, 100, 0, 0, 20/100, 21),
        trading_params = (1, 1/100),
        replay_memory_size = 600000,
        minibatch_size = 128,
        learning_rate = 1e-4,
        discount_factor = 0.99,
        risk_adverse_constant = 1.5,
        epsilon_params = (1.0, 0.9994, 0.01, 2500),
        episode_n = 50000,
        model_name = 'ActorCritic',
        stats_frequency = 1000,
        weight_save_frequency = 1000,
        target_model_update_frequency = 2500,
        load_flag = False,
        tag = ''
        ):

        # Environment
        self.env = ContinuousActionHedgingEnv(
            underlying_params,
            option_params,
            trading_params
        )

        # Replay Memory
        self.replay_memory = ReplayMemory(
            replay_memory_size,
            minibatch_size
        )

        # Network
        self.agent = ActorCritic(
            self.env,
            learning_rate ,
            discount_factor,
            risk_adverse_constant
        )
        self.model_name = model_name
        self.tag = tag
        self.load_flag = load_flag
        if load_flag:
            self.agent.load_weights(self.model_name, self.tag)

        # Algo
        self.epsilon, self.epsilon_decay, self.epsilon_min, self.epsilon_decay_start = epsilon_params

        self.episode_n = episode_n
        self.episode_count = 0
        self.step_count = 0

        self.stats_frequency = stats_frequency
        self.episode_final_reward_list = []
        self.stats = {}

        self.target_model_update_frequency = target_model_update_frequency
        self.weight_save_frequency = weight_save_frequency


    def epsilon_greedy_action_continuous_case(self, state):
        if np.random.rand() <= self.epsilon:
            action = self.env.action_space.sample().item()
        else:
            action = self.agent.actor.predict(state)[0][0]
            # [0][0] to have only the action and not an array
            action = np.clip(action, 0, 100)
        return action


    def epsilon_update(self):
        if self.epsilon >= self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    def get_stats(self):
        if not self.episode_final_reward_list:
            print('No stats for empty reward list.')
        else:
            self.stats['episode'] = self.episode_count
            self.stats['average'] = np.mean(self.episode_final_reward_list)
            self.stats['min'] = np.min(self.episode_final_reward_list)
            self.stats['max'] = np.max(self.episode_final_reward_list)
            self.stats['last'] = self.episode_final_reward_list[-1]
            print(f"Episode: {self.stats['episode']}\nAverage Final Wealth: {self.stats['average']}\nMin Final Wealth: {self.stats['min']}\nMax Final Wealth: {self.stats['max']}\nLast Final Wealth: {self.stats['last']}")


    def train(self):

        if self.load_flag:
            start = self.tag
        else:
            start = 1

        for episode in tqdm(range(start, self.episode_n + 1)): # 1 episode = 1 trajectory

            self.episode_count += 1

            # 3.
            # Initialize sequence
            state = self.env.reset()
            episode_reward_list = []

            while not self.env.done: # done when meeting end of trajectory

                self.step_count += 1

                # 4.
                # Select epsilon-greedy action
                action = self.epsilon_greedy_action_continuous_case(state)

                # 5. 6.
                # Execute action
                new_state, new_reward, done, info = self.env.step(action)

                # 7.
                # Store transition in replay memory
                self.replay_memory.add(state, action, new_reward, new_state, done)

                # 8.
                # Sample random minibatch
                if  len(self.replay_memory) > self.replay_memory.batch_size:
                    minibatch = self.replay_memory.sample()
                    self.agent.train(minibatch)
                    del minibatch

                # 9.
                # Reset Q_target every C steps
                if  self.step_count % self.target_model_update_frequency == 0 and self.step_count > self.replay_memory.batch_size:
                    self.agent.update_target_model()

                # Close loop
                state = new_state
                episode_reward_list.append(new_reward)


            # Decay epsilon. Only start after replay memory is over min size
            if self.step_count > self.epsilon_decay_start:
                self.epsilon_update()

            # Follow-up
            self.episode_final_reward_list.append(np.sum(episode_reward_list))
            if episode % self.stats_frequency == 0 and episode > 1:
                self.get_stats()

            # Save weights
            if episode % self.weight_save_frequency == 0:
                self.agent.save_weights(self.model_name, episode)

        self.agent.save_weights(self.model_name)


    def test(
        self,
        delta_action_flag = False,
        episode_n = 10000,
        verbose_frequency = 1000
        ):
        """Test should be run in another instance"""

        # Reinitialize
        self.epsilon = -1
        episode_index = 0

        for episode in tqdm(range(1, episode_n + 1)): # 1 episode = 1 trajectory

            episode_index += 1

            # Initialize
            state = self.env.reset()
            episode_action_list = []
            episode_reward_list = []

            while not self.env.done:

                # Select action
                if delta_action_flag:
                    action = self.env.option_deltas[self.env.index_trajectory,self.env.index_step] * 100
                else:
                    action = self.epsilon_greedy_action_continuous_case(tf.expand_dims(state, axis=0))

                # Take action
                new_state, new_reward, done, info = self.env.step(action)

                # Close loop
                episode_action_list.append(action)
                episode_reward_list.append(new_reward)
                state = new_state

            # Follow-up
            self.episode_final_reward_list.append(np.sum(episode_reward_list))

            if episode % verbose_frequency == 0:
                R_final_mean = np.mean(self.episode_final_reward_list)
                R_final_var = np.var(self.episode_final_reward_list)
                Y = -R_final_mean + self.agent.risk_adverse_constant*np.sqrt(R_final_var)

                print(f"\nEpisode {episode} final reward is: {self.episode_final_reward_list[-1]}")
                print(f"Final reward mean: {R_final_mean}")
                print(f"Final reward var: {R_final_var}")
                print(f"Y = {Y}")



### small run - train

In [17]:
# # Parameters 'small run'
# algo_ac = MainAC(
#     underlying_params = (100, 5/100, 20/100, 252, 50000),
#     option_params = (100, 100, 0, 0, 20/100, 21),
#     trading_params = (1, 1/100),
#     replay_memory_size = 6000,
#     minibatch_size = 64,
#     learning_rate = 1e-4,
#     discount_factor = 0.99,
#     risk_adverse_constant=1.5,
#     epsilon_params =(1.0, 0.99994, 0.01, 25),
#     episode_n = 500,
#     model_name = 'ActorCritic - small run',
#     stats_frequency = 10,
#     weight_save_frequency = 10,
#     target_model_update_frequency = 25
# )

# algo_ac.train()

### small run - test

In [ ]:
# Parameters 'small run'
# algo_ac_test = MainAC(
#     underlying_params = (100, 5/100, 20/100, 252, 50000),
#     option_params = (100, 100, 0, 0, 20/100, 21),
#     trading_params = (1, 1/100),
#     replay_memory_size = 6000,
#     minibatch_size = 64,
#     learning_rate = 1e-4,
#     discount_factor = 0.99,
#     risk_adverse_constant=1.5,
#     epsilon_params =(1.0, 0.99994, 0.01, 25),
#     episode_n = 500,
#     model_name = 'ActorCritic - small run',
#     stats_frequency = 10,
#     weight_save_frequency = 10,
#     target_model_update_frequency = 25,
#     load_flag = True,
#     tag = 40 # no tag means final weights
# )

# algo_ac_test.test(
#     delta_action_flag = False,
#     # delta_action_flag = True,
#     episode_n = 100,
#     verbose_frequency = 10
# )

# AC results (delta_action_flag = False)
# Episode 100 final reward is: -3.731407951438186
# Final reward mean: -3.528924405517586
# Final reward var: 31.959103145300347
# Y = 12.008781838100152

# Delta hedging (delta_action_flag = True)
# Episode 100 final reward is: -119.78820718706821
# Final reward mean: -202.08242112906365
# Final reward var: 122342.20017701518
# Y = 726.7441696854876

### full run

In [26]:
algo_train = MainAC()
algo_train.train()

algo_test = MainDQN(load_flag = True)
algo_test.test(delta_action_flag = False)
algo_test.test(delta_action_flag = True)

True